In [4]:
import pandas as pd
import numpy as np

np.random.seed(42)

# Product Master
product_data = {
    'ProductID': [101, 102, 103, 104, 105, 106, 107, 108],
    'ProductName': ['Laptop', 'Smartphone', 'Jeans', 'T-Shirt', 'Rice', 'Wheat', 'Headphones', 'Microwave'],
    'Category': ['Electronics', 'Electronics', 'Clothing', 'Clothing', 'Grocery', 'Grocery', 'Electronics', 'Electronics'],
    'UnitPrice': [75000, np.nan, 1500, 800, 1200, 1000, 2500, np.nan],
    'Stock': [10, 25, 60, 30, 80, 40, 15, 5]
}

df_products = pd.DataFrame(product_data)
df_products

,ProductID,ProductName,Category,UnitPrice,Stock
0,101,Laptop,Electronics,75000.0,10
1,102,Smartphone,Electronics,NaN,25
2,103,Jeans,Clothing,1500.0,60
3,104,T-Shirt,Clothing,800.0,30
4,105,Rice,Grocery,1200.0,80
5,106,Wheat,Grocery,1000.0,40
6,107,Headphones,Electronics,2500.0,15
7,108,Microwave,Electronics,NaN,5


In [5]:
sales_data = {
    'SaleID': [201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 210],  # Notice duplicate SaleID
    'ProductID': [101, 102, 103, 104, 105, 106, 107, 108, 101, 105, 105],  # 105 appears twice
    'QuantitySold': [1, 2, 3, 2, 5, 1, 4, 1, 2, 3, 3],
    'SaleDate': ['2025-06-10', '2025-06-10', '2025-06-11', '2025-06-12', '2025-06-12', 
                 '2025-06-13', '2025-06-13', '2025-06-14', '2025-06-14', '2025-06-15', '2025-06-15'], 
    'CustomerType': ['Regular', 'Member', 'Regular', 'Member', 'Regular', 
                     'Member', 'Member', 'Regular', 'Regular', 'Member', 'Member']
}

df_sales = pd.DataFrame(sales_data)
df_sales['SaleDate'] = pd.to_datetime(df_sales['SaleDate'])
df_sales

,SaleID,ProductID,QuantitySold,SaleDate,CustomerType
0,201,101,1,2025-06-10,Regular
1,202,102,2,2025-06-10,Member
2,203,103,3,2025-06-11,Regular
3,204,104,2,2025-06-12,Member
4,205,105,5,2025-06-12,Regular
5,206,106,1,2025-06-13,Member
6,207,107,4,2025-06-13,Member
7,208,108,1,2025-06-14,Regular
8,209,101,2,2025-06-14,Regular
9,210,105,3,2025-06-15,Member


In [6]:
df_products

,ProductID,ProductName,Category,UnitPrice,Stock
0,101,Laptop,Electronics,75000.0,10
1,102,Smartphone,Electronics,NaN,25
2,103,Jeans,Clothing,1500.0,60
3,104,T-Shirt,Clothing,800.0,30
4,105,Rice,Grocery,1200.0,80
5,106,Wheat,Grocery,1000.0,40
6,107,Headphones,Electronics,2500.0,15
7,108,Microwave,Electronics,NaN,5


In [ ]:
# Fill missing 'UnitPrice' with category-wise average.

df_products['UnitPrice'] = pd.to_numeric(df_products['UnitPrice'], errors='coerce')
df_products['UnitPrice'] = df_products.groupby('Category')['UnitPrice'].transform(
    lambda x: x.fillna(x.mean())
)
df_products

,ProductID,ProductName,Category,UnitPrice,Stock
0,101,Laptop,Electronics,75000.0,10
1,102,Smartphone,Electronics,38750.0,25
2,103,Jeans,Clothing,1500.0,60
3,104,T-Shirt,Clothing,800.0,30
4,105,Rice,Grocery,1200.0,80
5,106,Wheat,Grocery,1000.0,40
6,107,Headphones,Electronics,2500.0,15
7,108,Microwave,Electronics,38750.0,5


In [23]:
# Remove duplicate sales records.
df_sales.drop_duplicates(inplace=True)

# Replace 'Regular'/'Member' in 'CustomerType' with 'R' and 'M'
df_sales['CustomerType'] = df_sales['CustomerType'].replace({'Regular' : 'R', 'Member' : 'M'})
df_sales

,SaleID,ProductID,QuantitySold,SaleDate,CustomerType
0,201,101,1,2025-06-10,R
1,202,102,2,2025-06-10,M
2,203,103,3,2025-06-11,R
3,204,104,2,2025-06-12,M
4,205,105,5,2025-06-12,R
5,206,106,1,2025-06-13,M
6,207,107,4,2025-06-13,M
7,208,108,1,2025-06-14,R
8,209,101,2,2025-06-14,R
9,210,105,3,2025-06-15,M


In [66]:
# Merge Sales Data with Product Master on 'ProductID'.
merged_df = pd.merge(df_products, df_sales, on=['ProductID'], how='left')

# Calculate 'TotalSaleAmount' = 'UnitPrice' * 'QuantitySold' in the merged DataFrame
merged_df['Total Amount'] = merged_df['UnitPrice'] * merged_df['QuantitySold']

# Find Most Selling Product (by Total Quantity Sold).
most_selling_product = merged_df.groupby('ProductName')['QuantitySold'].sum().idxmax()

# Find the Category with Maximum Total Sales Amount.
category_max_sales = merged_df.groupby('Category')['Total Amount'].sum().idxmax()
category_max_sales, most_selling_product

('Electronics', 'Rice')

In [ ]:
# Filter Sales where 'QuantitySold' > 2 and 'Category' is 'Electronics'.
merged_df_filter = merged_df[(merged_df['QuantitySold'] > 2) & (merged_df['Category'] == 'Electronics')] 


,ProductID,ProductName,Category,UnitPrice,Stock,SaleID,QuantitySold,SaleDate,CustomerType,Total Amount
8,107,Headphones,Electronics,2500.0,15,207,4,2025-06-13,M,10000.0


In [65]:
# Group by 'ProductName' and calculate Total Sales Quantity and Total Sales Amount.
merged_df
product_sales_summary = merged_df.groupby('ProductName').agg(
    TotalQuantitySold=('QuantitySold', 'sum'),
    TotalSaleAmount=('Total Amount', 'sum')
)
merged_df

,ProductID,ProductName,Category,UnitPrice,Stock,SaleID,QuantitySold,SaleDate,CustomerType,Total Amount,StockCategory
0,101,Laptop,Electronics,75000.0,NaN,201,1,2025-06-10,R,75000.0,NaN
1,101,Laptop,Electronics,75000.0,NaN,209,2,2025-06-14,R,150000.0,NaN
2,102,Smartphone,Electronics,38750.0,NaN,202,2,2025-06-10,M,77500.0,NaN
3,103,Jeans,Clothing,1500.0,NaN,203,3,2025-06-11,R,4500.0,NaN
4,104,T-Shirt,Clothing,800.0,NaN,204,2,2025-06-12,M,1600.0,NaN
5,105,Rice,Grocery,1200.0,NaN,205,5,2025-06-12,R,6000.0,NaN
6,105,Rice,Grocery,1200.0,NaN,210,3,2025-06-15,M,3600.0,NaN
7,106,Wheat,Grocery,1000.0,NaN,206,1,2025-06-13,M,1000.0,NaN
8,107,Headphones,Electronics,2500.0,NaN,207,4,2025-06-13,M,10000.0,NaN
9,108,Microwave,Electronics,38750.0,NaN,208,1,2025-06-14,R,38750.0,NaN


In [48]:
# Group by 'SaleDate' and find the day with Maximum Total Sales.
sales_df = merged_df.groupby('SaleDate')['Total Amount'].sum().idxmax()
sales_df

Timestamp('2025-06-14 00:00:00')

In [67]:
# Sort Products by 'Stock' descending — to find low-stock products.
sorted_products = merged_df.sort_values(by='Stock', ascending=False)
sorted_products

,ProductID,ProductName,Category,UnitPrice,Stock,SaleID,QuantitySold,SaleDate,CustomerType,Total Amount
6,105,Rice,Grocery,1200.0,80,210,3,2025-06-15,M,3600.0
5,105,Rice,Grocery,1200.0,80,205,5,2025-06-12,R,6000.0
3,103,Jeans,Clothing,1500.0,60,203,3,2025-06-11,R,4500.0
7,106,Wheat,Grocery,1000.0,40,206,1,2025-06-13,M,1000.0
4,104,T-Shirt,Clothing,800.0,30,204,2,2025-06-12,M,1600.0
2,102,Smartphone,Electronics,38750.0,25,202,2,2025-06-10,M,77500.0
8,107,Headphones,Electronics,2500.0,15,207,4,2025-06-13,M,10000.0
0,101,Laptop,Electronics,75000.0,10,201,1,2025-06-10,R,75000.0
1,101,Laptop,Electronics,75000.0,10,209,2,2025-06-14,R,150000.0
9,108,Microwave,Electronics,38750.0,5,208,1,2025-06-14,R,38750.0


In [54]:
top_3_products = merged_df.groupby('ProductName')['Total Amount'].sum().nlargest(3)
top_3_products

ProductName
Laptop        225000.0
Smartphone     77500.0
Microwave      38750.0
Name: Total Amount, dtype: float64

In [70]:
def stock_level(x):
    if x >= 50:
        return 'High'
    elif 20 <= x <= 49:
        return 'Medium'
    else:
        return 'Low'

merged_df['StockLevel'] = merged_df['Stock'].apply(stock_level)
merged_df

,ProductID,ProductName,Category,UnitPrice,Stock,SaleID,QuantitySold,SaleDate,CustomerType,Total Amount,StockLevel
0,101,Laptop,Electronics,75000.0,10,201,1,2025-06-10,R,75000.0,Low
1,101,Laptop,Electronics,75000.0,10,209,2,2025-06-14,R,150000.0,Low
2,102,Smartphone,Electronics,38750.0,25,202,2,2025-06-10,M,77500.0,Medium
3,103,Jeans,Clothing,1500.0,60,203,3,2025-06-11,R,4500.0,High
4,104,T-Shirt,Clothing,800.0,30,204,2,2025-06-12,M,1600.0,Medium
5,105,Rice,Grocery,1200.0,80,205,5,2025-06-12,R,6000.0,High
6,105,Rice,Grocery,1200.0,80,210,3,2025-06-15,M,3600.0,High
7,106,Wheat,Grocery,1000.0,40,206,1,2025-06-13,M,1000.0,Medium
8,107,Headphones,Electronics,2500.0,15,207,4,2025-06-13,M,10000.0,Low
9,108,Microwave,Electronics,38750.0,5,208,1,2025-06-14,R,38750.0,Low


In [79]:
# 16. Pivot Table: Category vs CustomerType — Total QuantitySold
pivot = pd.pivot_table(
    merged_df,
    index='Category',
    columns='CustomerType',
    values='QuantitySold',
    aggfunc='sum',
    fill_value=0
)

# 17. Reindex Product Master with index starting from 1000
df_products.index = range(1000, 1000 + len(df_products))


In [84]:
# 18. Convert Merged DataFrame to list of dictionaries
merged_dict_list = merged_df.to_dict(orient='records')
print(merged_dict_list)

# 19. Export Sales Summary (Product, TotalSaleAmount) as CSV string
sales_summary = merged_df.groupby('ProductName')['Total Amount'].sum().reset_index()
csv_output = sales_summary.to_csv(index=False)
print(csv_output)

# 20. Product with Maximum TotalSaleAmount
max_sale_product = sales_summary.loc[sales_summary['Total Amount'].idxmax(), 'ProductName']


[{'ProductID': 101, 'ProductName': 'Laptop', 'Category': 'Electronics', 'UnitPrice': 75000.0, 'Stock': 10, 'SaleID': 201, 'QuantitySold': 1, 'SaleDate': Timestamp('2025-06-10 00:00:00'), 'CustomerType': 'R', 'Total Amount': 75000.0, 'StockLevel': 'Low'}, {'ProductID': 101, 'ProductName': 'Laptop', 'Category': 'Electronics', 'UnitPrice': 75000.0, 'Stock': 10, 'SaleID': 209, 'QuantitySold': 2, 'SaleDate': Timestamp('2025-06-14 00:00:00'), 'CustomerType': 'R', 'Total Amount': 150000.0, 'StockLevel': 'Low'}, {'ProductID': 102, 'ProductName': 'Smartphone', 'Category': 'Electronics', 'UnitPrice': 38750.0, 'Stock': 25, 'SaleID': 202, 'QuantitySold': 2, 'SaleDate': Timestamp('2025-06-10 00:00:00'), 'CustomerType': 'M', 'Total Amount': 77500.0, 'StockLevel': 'Medium'}, {'ProductID': 103, 'ProductName': 'Jeans', 'Category': 'Clothing', 'UnitPrice': 1500.0, 'Stock': 60, 'SaleID': 203, 'QuantitySold': 3, 'SaleDate': Timestamp('2025-06-11 00:00:00'), 'CustomerType': 'R', 'Total Amount': 4500.0, 'St